In [ ]:
# draw function on 2d elements

import webgpu.jupyter
#from webgpu.mesh import MeshData
from ngsolve import *

def render(data):
    import ngsolve_webgpu.mesh
    import pyodide.ffi
    import webgpu
    from webgpu.jupyter import gpu
    import ngsolve as ngs
    import js

    mesh = data["mesh"]
    cf = data["cf"]
    order = data.get("order", 1)

    mesh_data = ngsolve_webgpu.mesh.MeshData(mesh, cf, order)
    mesh_object = ngsolve_webgpu.mesh.MeshRenderObject(gpu, mesh_data)
    point_numbers = ngsolve_webgpu.mesh.PointNumbersRenderObject(gpu, mesh_data, 13)
    gpu.colormap.set_min_max(0., 1.)
    def render_function(t):
        gpu.update_uniforms()
        # gpu.uniforms.update_buffer()

        encoder = gpu.device.createCommandEncoder()
        mesh_object.render(encoder)
        point_numbers.render(encoder)
        gpu.device.queue.submit([encoder.finish()])

    render_function = pyodide.ffi.create_proxy(render_function)
    gpu.input_handler.render_function = render_function
    js.requestAnimationFrame(render_function)

m = Mesh(unit_square.GenerateMesh(maxh=.1))
#m = Mesh(unit_cube.GenerateMesh(maxh=.3))
cf=sin(10*x)*sin(10*y)

# "data" will be passed as argument to render on the pyodide side
webgpu.jupyter.DrawCustom({"cf": cf, "mesh": m, "order": 5}, render, ["ngsolve_webgpu"])

In [ ]:
# draw geometry

import webgpu.jupyter
from netgen.occ import *
from ngsolve import *

box = Box((-1,-1,-1), (1,1,1)) + Sphere((1,1,1), 0.4)

def render(data):
    import ngsolve_webgpu.geometry
    ngsolve_webgpu.geometry.render_geometry(data["geo"])

def draw_geo(geo):
    if isinstance(geo, TopoDS_Shape):
        geo = OCCGeometry(geo)
    webgpu.jupyter.DrawCustom({"geo" : geo}, render, ["ngsolve_webgpu"])

draw_geo(box)

In [ ]:
# draw isosurface

import webgpu.jupyter
from netgen.occ import *
from ngsolve import *

box = Box((-1,-1,-1), (1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=.15))

cf = x**2 + y**2 + z**2 - 0.5**2

def render(data):
    import ngsolve_webgpu.isosurface
    ngsolve_webgpu.isosurface.render_isosurface(data["levelset"], data["func"], data["mesh"])

def draw_iso(levelset, func, mesh):
    webgpu.jupyter.DrawCustom({"levelset" : levelset, "mesh" : mesh, "func" : func}, render, ["ngsolve_webgpu"])

draw_iso(cf, x/Norm(CF((x,y,z))), mesh)